<a href="https://colab.research.google.com/github/Khang2561/Intrusion-detection-using-DL-and-MC/blob/main/mordeling_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Read file


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
#read file train and test
#train
path = "/content/drive/MyDrive/predict_test/kdd_train.csv"
train_df = pd.read_csv(path)

#test
path = "/content/drive/MyDrive/predict_test/kdd_test.csv"
test_df = pd.read_csv(path)

In [38]:
#Phân loại những data có kiểu category
#change Dtype
features = ["protocol_type","service","flag","land","wrong_fragment","num_outbound_cmds","is_host_login","urgent","logged_in","root_shell","su_attempted","is_guest_login","labels"]
def convert_cat(df,features) :
  for feature in features :
    df[feature] = df[feature].astype("category")
convert_cat(train_df, features)
convert_cat(test_df, features)

In [39]:
#include = [col for col in train_df.columns if train_df[col].dtype != 'category']
#include
num_features = ['duration',
 'src_bytes',
 'dst_bytes',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']
cat_features = ["protocol_type","service","flag","land","wrong_fragment","num_outbound_cmds","is_host_login","urgent","logged_in","root_shell","su_attempted","is_guest_login","labels"]
#preprocessor = ColumnTransformer(transformers=[
#    ('num', num_transformer, num_features),
#    ('cat', cat_transformer, cat_features)
#])

In [40]:
feature_cols = num_features +cat_features
x = train_df[feature_cols]
y = train_df['labels']
X_test =test_df[feature_cols]
y_test = test_df['labels']

In [41]:
#preprocess pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
num_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [42]:
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

In [43]:
preprocessor.fit(x)
#
preprocessor.fit(X_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['duration', 'src_bytes', 'dst_bytes', 'hot',
                                  'num_failed_logins', 'num_compromised',
                                  'num_root', 'num_file_creations',
                                  'num_shells', 'num_access_files', 'count',
                                  'srv_count', 'serror_rate', 'srv_serror_rate',
                                  'rerror_rate', 'srv_rerro...
                                  'dst_host_rerror_rate',
                                  'dst_host_srv_rerror_rate']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['protocol_type', 'service', 'flag', 'land',
                                  'wrong_fragment', 'num_outbound_cmds',
                                  'is_host_login', 'urgent', 'logged_in',
                                  'root_shell', 'su_attempted',
                                  'is_guest_login', 'labels'])])

In [44]:
X = preprocessor.transform(x)#X -> X_train, X_val

In [45]:
X_test =preprocessor.transform(X_test)

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_val, y_train,y_val = train_test_split(X,y, test_size = 0.2)

In [ ]:
X_train.shape, X_val.shape

((100778, 157), (25195, 157))

In [ ]:
X_test.shape

(22544, 157)

In [51]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
#binary classification
from sklearn.preprocessing import PolynomialFeatures
#sklearn model
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score,classification_report, confusion_matrix


Liblinear


In [52]:
log_reg = LogisticRegression(solver = 'liblinear',max_iter = 1000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [53]:
log_reg.score(X_val, y_val)

0.9999206191704704

In [54]:
log_reg.score(X_test,y_test)

0.9303140525195174

DecisionTreeClassifier

In [55]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

tree.score(X_val,y_val)


1.0

In [56]:
tree.score(X_test,y_test)

0.94916607523066

RandomForestClassifier

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=1000, random_state=0)
clf.fit(X_train, y_train)

clf.score(X_val,y_val)


0.9998015479261758

In [58]:
clf.score(X_test,y_test)

0.9204666430092264

ExtraTreesClassifier

In [49]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification

clf = ExtraTreesClassifier(n_estimators=1000, random_state=0)
clf.fit(X_train, y_train)

clf.score(X_val,y_val)


0.9998412383409406

In [50]:
clf.score(X_test,y_test)


0.9316004258339248

SGD Classifier

In [59]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [60]:
SGD = SGDClassifier(loss="perceptron", eta0=1,max_iter=1000)

In [61]:
SGD.fit(X_train, y_train)

SGDClassifier(eta0=1, loss='perceptron')

In [62]:
SGD.score(X_val, y_val)

0.9923000595356222

In [63]:
SGD.score(X_test,y_test)

0.910929737402413

Kernel Approximation

In [64]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

In [65]:
rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X_train)
clf = SGDClassifier(max_iter=10000)
clf.fit(X_train, y_train)
SGDClassifier(max_iter=10000)
clf.score(X_val, y_val)

0.9945624131772177

In [66]:
clf.score(X_test,y_test)

0.9149662881476224

Navie bayes


In [67]:
from sklearn.naive_bayes import GaussianNB

In [68]:
NB = GaussianNB()
X_train_dense = X_train.toarray()
NB.fit(X_train_dense, y_train)

GaussianNB()

In [69]:
X_val_dense = X_val.toarray()
NB.score(X_val_dense, y_val)

0.9998015479261758

In [71]:
X_test_dense = X_test.toarray()
NB.score(X_test_dense,y_test)

0.9306689141234918

KNeighborsClassifier

In [72]:
from sklearn.neighbors import KNeighborsClassifier

In [73]:
neigh = KNeighborsClassifier(n_neighbors=1000)#Train nhanh ra quyet dinh lau
X_train_dense = X_train.toarray()
neigh.fit(X_train_dense, y_train)

KNeighborsClassifier(n_neighbors=1000)

In [74]:
X_val_dense = X_val.toarray()
neigh.score(X_val_dense, y_val)

0.9611033935304624

In [75]:
X_test_dense = X_test.toarray()
neigh.score(X_test_dense,y_test)

0.8744677075940384

SVC


In [76]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
X_train_dense = X_train.toarray()
clf.fit(X_train_dense, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [77]:
X_val_dense = X_val.toarray()
clf.score(X_val_dense, y_val)

0.9993252629489978

In [78]:
X_test_dense = X_test.toarray()
clf.score(X_test_dense,y_test)

0.9364354151880766

Ensembles

In [79]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2

In [80]:
X_train_dense = X_train.toarray()
clf = HistGradientBoostingClassifier(max_iter=100).fit(X_train_dense, y_train)


In [81]:
X_val_dense = X_val.toarray()
clf.score(X_val_dense, y_val)

0.9122444929549514

In [82]:
X_test_dense = X_test.toarray()
clf.score(X_test_dense,y_test)

0.8479418026969482

Clear data


In [83]:
#xóa những cột không quá ảnh hưởng đến bài trên train_df
train_df = train_df.drop(labels ='num_outbound_cmds',axis = 1)
train_df = train_df.drop(labels ='is_host_login',axis = 1)


In [84]:
#include = [col for col in train_df.columns if train_df[col].dtype != 'category']
#include
num_features = ['duration',
 'src_bytes',
 'dst_bytes',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']
cat_features =  ["protocol_type","service","flag","land","wrong_fragment","urgent","logged_in","root_shell","su_attempted","is_guest_login","labels"]
#preprocessor = ColumnTransformer(transformers=[
#    ('num', num_transformer, num_features),
#    ('cat', cat_transformer, cat_features)
#])


In [85]:
feature_cols = num_features +cat_features
x = train_df[feature_cols]
y = train_df['labels']
X_test =test_df[feature_cols]
y_test = test_df['labels']

In [86]:
#preprocess pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [87]:
num_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [88]:
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

In [89]:
preprocessor.fit(x)
preprocessor.fit(X_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['duration', 'src_bytes', 'dst_bytes', 'hot',
                                  'num_failed_logins', 'num_compromised',
                                  'num_root', 'num_file_creations',
                                  'num_shells', 'num_access_files', 'count',
                                  'srv_count', 'serror_rate', 'srv_serror_rate',
                                  'rerror_rate', 'srv_rerro...
                                  'dst_host_serror_rate',
                                  'dst_host_srv_serror_rate',
                                  'dst_host_rerror_rate',
                                  'dst_host_srv_rerror_rate']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['protocol_type', 'service', 'flag', 'land',
                                  'wrong_fragment', 'urgent', 'logged_in',
                                  'root_shell', 'su_attempted',
                                  'is_guest_login', 'labels'])])

In [90]:
X = preprocessor.transform(x)#X -> X_train, X_val

In [91]:
X_test =preprocessor.transform(X_test)

In [92]:
from sklearn.model_selection import train_test_split

In [93]:
X_train, X_val, y_train,y_val = train_test_split(X,y, test_size = 0.2)

In [94]:
log_reg = LogisticRegression(solver = 'liblinear',max_iter = 1000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [95]:
log_reg.score(X_val, y_val)

0.9998015479261758

In [97]:
log_reg.score(X_test,y_test)


0.9298704755145494